In [2]:
print("Hello World")

Hello World


In [3]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from scipy import spatial
import numpy as np
import cv2
import os
from torch.utils.data import Dataset, DataLoader
import random
import shutil
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [4]:
resnet_model = models.resnet50(pretrained=True)
resnet_model.eval()
resnet_model = torch.nn.Sequential(*(list(resnet_model.children())[:-1]))
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225]),
])

C:\Users\97433\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\97433\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
def softmax ( x : float ) -> float: 
    """
        Returns output after applying softmax function on input x. 
    """
    return np.exp ( x ) / np.sum ( np.exp ( x ) )

In [6]:
def get_normalized_feat_vec ( image_path : str, 
                  transform = transform,
                  model = resnet_model):
    """
        Returns a feature vector for the image stored 
        at image path using pretrained model.
    """
    image = Image.open ( image_path )
    input_image = transform(image).unsqueeze(0)  
    with torch.no_grad():
        output = model(input_image)
    feat_vec_as_np_array = output.squeeze().numpy()
    normalized_feat_vec_as_np_array = softmax ( feat_vec_as_np_array )
    return normalized_feat_vec_as_np_array

In [7]:
def get_cosine_similarity_between_vectors ( first_feat_vec : list,
                                        second_feat_vec : list ) -> float:
    """
        Returns cosine similarity score between two feature vectors.
    """
    return 1 - spatial.distance.cosine( first_feat_vec, second_feat_vec)

In [8]:
def get_euclidean_distance ( first_feat_vec : list, 
                            second_feat_vec : list ) -> float:
    """
        Returns euclidean distance between two feature vectors.
    """
    return spatial.distance.euclidean ( first_feat_vec, second_feat_vec )

In [9]:
def get_cosine_similarity_score ( first_image_path : str , second_image_path : str,
                                transform = transform , model = resnet_model ) -> float:
    """
        Returns cosine similarity score between two images based 
        on their feature vectors obatined from pretrained models.
    """    
    first_image_feat_vec = get_normalized_feat_vec ( first_image_path , transform , model )
    second_image_feat_vec = get_normalized_feat_vec ( second_image_path , transform , model )

    return get_cosine_similarity_between_vectors ( first_image_feat_vec, 
                                                  second_image_feat_vec)

In [10]:
def get_histogram_dissimilarity_score ( first_feat_vec, second_feat_vec ):
    return sum(abs((first_feat_vec - second_feat_vec) / ( 1 + first_feat_vec + second_feat_vec )))

In [11]:
def get_euclidean_dissimilarity_score ( first_image_path : str , second_image_path : str ,
                            transform = transform , model = resnet_model ) -> float:
    """
        Returns dissimilarity score between two images based on the euclidean distance 
        between their feature vectors obatined from pretrained models.
    """    
    first_image_feat_vec = get_normalized_feat_vec ( first_image_path , transform , model )
    second_image_feat_vec = get_normalized_feat_vec ( second_image_path ,transform , model )

    return get_euclidean_distance ( first_image_feat_vec , second_image_feat_vec )

In [12]:
image1 = r"C:\Users\97433\Mafkin_Robotics_FrontEnd\VisualOdometry\feature_extraction\dataset\blobs\image_pair1\blob0\image2\blob_at_2448_1924_image_2.jpg"
image2 = r"C:\Users\97433\Mafkin_Robotics_FrontEnd\VisualOdometry\feature_extraction\dataset\blobs\image_pair1\blob0\image1\blob_at_1573_2187_image_1.jpg"

In [13]:
def pad_image ( image : list, border_len = 1 ):
    """
        Pads image with 0s such that length and height
        of image increase by border_len.
    """
    return np.pad ( image , border_len, mode = "constant" )

In [14]:
def F1( intensity : int):
    return int ( intensity >= 0 )

def F3 ( k1 : int, k2 : int ):
    is_k1_non_negative = k1 >= 0
    is_k2_non_negative = k2 >= 0

    return ~(is_k1_non_negative ^ is_k2_non_negative) & 1

def labeled_coord_of_patch ( n : int, x : int , y : int ):
    """
        Returns coordinates (x,y) of image according to 
        label number n
    """
    try:
        labeled_coords = [(0,0), (0,1), (1,1), 
                          (1,0), (1,-1), (0,-1), 
                          (-1,-1), (-1,0), (-1,1) ]
        coord = labeled_coords [ n ]
        #print ( coord [0] + x , coord [1] + y )
        return ( coord [0] + x , coord [1] + y )
    except Exception as e:
        raise ValueError("Provide x and y between 0 and 8") from e

In [15]:
def labeled_coord_of_patch ( n : int, x : int , y : int ):
    """
        Returns coordinates (x,y) of image according to 
        label number n
    """
    try:
        labeled_coords = [(0,0), (0,1), (1,1), 
                          (1,0), (1,-1), (0,-1), 
                          (-1,-1), (-1,0), (-1,1) ]
        coord = labeled_coords [ n ]
        #print ( coord [0] + x , coord [1] + y )
        return ( coord [0] + x , coord [1] + y )
    except Exception as e:
        raise ValueError("Provide x and y between 0 and 8") from e

def LBP( image : list, x : int, y : int ):
    """
        Performs Local Binary Pattern.
        Returns binary descriptor for patch whose center coordinates are given by input x and y. 
    """
    #print ( x , y )
    return sum([(2 ** (i-1)) * \
    F1(image[labeled_coord_of_patch(i,x,y)[0]][labeled_coord_of_patch(i,x,y)[1]] \
       - image[labeled_coord_of_patch(0,x,y)[0]][labeled_coord_of_patch(0,x,y)[1]]) \
    for i in range ( 1 , 9 ) ])

def LNDP ( image : list, x : int, y : int ):
    """
        Performs Local Neighbourhood Pattern.
        Returns binary descriptor for patch whose center coordinates are given by input x and y. 
    """
    def calculate_k1 ( n : int ):
        m = n - 1
        coords_for_In = labeled_coord_of_patch ( n , x,  y )

        if n == 1:
            coords_for_I8 = labeled_coord_of_patch ( 8 , x , y ) 
            #print ( coords_for_I8 )
            return image [ coords_for_I8[0] ][ coords_for_I8[1] ] - image [ coords_for_In[0] ][ coords_for_In[1] ]
        elif 2 <= n <= 8:
            coords_for_Im = labeled_coord_of_patch ( m , x, y )
            #print(coords_for_Im, coords_for_In)
            return image [ coords_for_Im[0]][ coords_for_Im[1] ] - image [ coords_for_In[0]][ coords_for_In[1] ]

    def calculate_k2 ( n : int ):
        l = n + 1
        coords_for_In = labeled_coord_of_patch ( n , x,  y )
        if 1 <= n <= 7:
            coords_for_Il = labeled_coord_of_patch ( l , x, y )
            return image [ coords_for_Il[0] ][ coords_for_Il[1] ] - image [ coords_for_In[0] ][ coords_for_In[1] ]
        elif n == 8:
            coords_for_I1 = labeled_coord_of_patch ( 1 , x , y )
            return image [ coords_for_I1[0] ][ coords_for_I1[1] ] - image [ coords_for_In[0] ][ coords_for_In[1] ]
        
    #print ( [ ( calculate_k1 ( i ) , calculate_k2 ( i ) ) for i in range ( 1, 9 )] )
    
    return sum([ 2 ** (i-1) * F3( calculate_k1 ( i ) , calculate_k2 ( i ) ) for i in range ( 1, 9 )])


In [16]:
def get_feature_vector_of_image_using_local_pattern_descriptor ( image : list, LPD ):
    """
        Gives feature vector for the image obtained by specified local pattern descriptor like 
        LBP ( Local Binary Pattern ) and LNDP ( Local Neighbourhood Pattern ).
    """
    h , w = image.shape
    padded_image = pad_image ( image )
    LPD_of_each_pixel = [ LPD ( padded_image , 
                               x + 1, y + 1 ) \
                            for x in range ( h ) \
                            for y in range ( w ) ]
    #print ( LPD_of_each_pixel )
    hist, _ = np.histogram ( LPD_of_each_pixel, 
                            bins = 256, range = ( 0 , 256 ) )

    return np.array(hist)

In [17]:
def get_normalized_vec( data : list, min_val : int, max_val : int) -> list:
    """
    Perform Min-Max normalization on the input data.

    Parameters:
    - data: A numpy array or list containing the data to be normalized.

    Returns:
    - normalized_data: The normalized data.
    """

    normalized_data = (data - min_val) / (max_val - min_val)
    return normalized_data

In [18]:
def normalize ( x : int, min_val : int, max_val : int ) -> int:
    return ( x - min_val ) / ( max_val - min_val )

In [19]:
def get_normalized_feat_vect_of_image_using_LBP_LNDP ( image : list ):
    """
        Gives feature vector for the image obtained by concatenating histograms of image obtained by
        LBP (Local Binary Pattern) and LNDP (Local Neighbourhood Pattern).
    """
    feat_vec = get_feature_vector_of_image_using_local_pattern_descriptor ( image, LBP ) +\
        get_feature_vector_of_image_using_local_pattern_descriptor ( image, LNDP )

    return get_normalized_vec ( feat_vec, 0, 256 )

In [20]:
def get_dissimilarity_score_for_two_images ( first_image_path, second_image_path, 
                                            disimilarity_func =\
                                            get_histogram_dissimilarity_score ) -> float:
    """
        Gives dissimilarity score between two images with the help of their feature vectors computed using LBP + LNDP
        technique.
    """

    first_image = cv2.imread ( first_image_path, 0 )
    second_image = cv2.imread ( second_image_path, 0 )

    first_image = cv2.resize ( first_image, ( 128 , 128 ) )
    second_image = cv2.resize ( second_image, ( 128 , 128 ) )

    first_image = first_image.astype ( np.int64 )
    second_image = second_image.astype ( np.int64 )

    dissimilarity_score =  disimilarity_func ( 
                get_normalized_feat_vect_of_image_using_LBP_LNDP( first_image ),
                get_normalized_feat_vect_of_image_using_LBP_LNDP ( second_image ) )

    return normalize ( dissimilarity_score, 0, 256 )


In [21]:
def get_blob(image : list, key_point : tuple , blob_size : tuple):
    """
    Extracts a patch (blob) from the image centered at the key_point
    with the specified blob_size (height, width).

    Args:
    - image: The input image.
    - key_point: The key point (x, y) around which the blob is extracted.
    - blob_size: The size of the blob in pixels (height, width).

    Returns:
    - blob_patch: The extracted patch (blob) from the image.
    """

    kp_x, kp_y = int(key_point[0]), int(key_point[1])

    blob_height, blob_width = blob_size

    top_left_x = max(0, kp_x - blob_width // 2)
    top_left_y = max(0, kp_y - blob_height // 2)

    bottom_right_x = min(image.shape[1], kp_x + blob_width // 2)
    bottom_right_y = min(image.shape[0], kp_y + blob_height // 2)

    blob_patch = image[top_left_y:bottom_right_y, top_left_x:bottom_right_x]

    return blob_patch

In [22]:
def get_blob_sequence_at_point ( image : list, coord : tuple, 
                               blob_sizes = [ 10, 25, 50, 100 ] ) -> list:
    """
    Parameters:
    - image (list): Image from which squence of blobs is to be generated.
    - coord (tuple): A tuple containing the (x, y) coordinate of the center of each blob.
    - blob_sizes (list): Sizes of blobs of the sequence.
    Returns:
    - blob_sequence (list): A list containing the blobs.
                            Each blob is represented as a square patch
                            of the specified size with its center
                            as given by the center_coordinates.
    """
    return [ get_blob ( image, coord, ( blob_size, blob_size)) \
            for blob_size in blob_sizes ]

In [23]:
image_path = r"C:\Users\97433\Mafkin_Robotics_FrontEnd\VisualOdometry\feature_extraction\dataset\images\image_pair1\image1.jpg"
image = cv2.imread ( image_path )
coord = ( image.shape[0] // 2, image.shape[1] // 2 )

In [24]:
def get_matching_keypoints( first_image_path : list, second_image_path : list) -> list:
    """ 
        Returns list of corresponding keypoints in both images sing ORB_FLANN feature matching.
    """ 
    first_image = cv2.imread ( first_image_path , 0 )
    second_image = cv2.imread ( second_image_path , 0 )
    
    orb = cv2.ORB_create()

    first_image_keypoints, first_descriptors = orb.detectAndCompute( first_image , None)
    second_image_keypoints, second_descriptors = orb.detectAndCompute( second_image , None)

    FLANN_INDEX_LSH = 6
    index_params = dict(algorithm=FLANN_INDEX_LSH, table_number=6, key_size=12, multi_probe_level=1)
    search_params = dict(checks=50)  

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch( first_descriptors, second_descriptors, k=2)

    good_matches = []
    for match_pair in matches:
        if len(match_pair) < 2:
            continue
        m , n = match_pair
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    first_image_matched_keypoints = [ (int(first_image_keypoints[match.queryIdx].pt[0]) ,
                                      int(first_image_keypoints[match.queryIdx].pt[1]))  \
                                    for match in good_matches]
    second_image_matched_keypoints = [ (int(second_image_keypoints[match.trainIdx].pt[0]),
                                        int(second_image_keypoints[match.trainIdx].pt[1])) \
                                      for match in good_matches]

    return [ [keypoint, second_image_matched_keypoints[index] ] \
            for index,keypoint in enumerate ( first_image_matched_keypoints )]


In [25]:
first_image_path = r"C:\Users\97433\Mafkin_Robotics_FrontEnd\VisualOdometry\feature_extraction\dataset\images\image_pair1\image1.jpg"
second_image_path = r"C:\Users\97433\Mafkin_Robotics_FrontEnd\VisualOdometry\feature_extraction\dataset\images\image_pair1\image2.jpg"
#get_matching_keypoints ( first_image_path, second_image_path )

In [26]:
def get_non_matching_keypoints ( first_image_path : str, second_image_path : str ) -> list:
    """
        Returns corresponding keypoints between two images.
    """
    orb_keypoints = get_matching_keypoints ( first_image_path, second_image_path )
    return [ [ first_image_keypoint[ 0 ], orb_keypoints [::-1] [ index ][ 1 ]] \
            for index, first_image_keypoint in enumerate ( orb_keypoints ) 
            if first_image_keypoint[ 1 ] != orb_keypoints [ ::-1 ][ index ][ 1 ] ]

In [27]:
def save_matching_keypoints_blob_sequences ( folder_to_save : str, 
                                            image_pair_folder : str,
                                            generate_keypoints_func : str,
                                            generate_blob_sequence_func = get_blob_sequence_at_point,
                                            extended_name_of_blob_sequence_folder = None) -> None:
    """
        Functionality : 
        - Saves blob sequence consisting of blobs of various sizes having keypoints as center
        for both images in the specified image pair folder. 
        
        Parameters : 
        - folder_to_save : folder path to save blob_sequence.
        - image_pair_folder : folder path for image pair.
        - generate_keypoints_function : function used to generate keypoints.
        - generate_blob_sequence_func : function used to generate blob sequence.
    """

    if not os.path.join ( folder_to_save ): os.makedirs ( folder_to_save )

    first_image_path, second_image_path =\
        [ os.path.join ( image_pair_folder, image )\
        for image in os.listdir( image_pair_folder ) ]
    
    list_of_keypoints = generate_keypoints_func ( first_image_path, second_image_path )

    first_image = cv2.imread ( first_image_path )
    second_image = cv2.imread ( second_image_path )

    blob_sequences_for_first_image = [ generate_blob_sequence_func ( first_image, keypoint[0] )\
                                      for keypoint in list_of_keypoints ]
    
    blob_sequences_for_second_image = [ generate_blob_sequence_func ( second_image, keypoint[1] )\
                                      for keypoint in list_of_keypoints ]

    for index, blob_sequence in enumerate ( blob_sequences_for_first_image ):
        blob_sequence_folder_name = "blob_sequence_" + str ( index ) + extended_name_of_blob_sequence_folder
        blob_sequence_folder_path = os.path.join ( folder_to_save, blob_sequence_folder_name )
        
        if not os.path.exists ( blob_sequence_folder_path ): os.makedirs ( blob_sequence_folder_path )

        first_image_blob_sequence_folder_path = os.path.join ( blob_sequence_folder_path, "image1" )
        second_image_blob_sequence_folder_path = os.path.join ( blob_sequence_folder_path, "image2" )

        if not os.path.exists ( first_image_blob_sequence_folder_path ): 
            os.makedirs ( first_image_blob_sequence_folder_path )

        if not os.path.exists ( second_image_blob_sequence_folder_path ):
            os.makedirs ( second_image_blob_sequence_folder_path )

        for blob_index, blob_image in enumerate ( blob_sequence ):

            blob_image_name = "blob" + str(blob_index) + ".jpg"
            
            blob_image_path = os.path.join ( first_image_blob_sequence_folder_path, 
                                                        blob_image_name )
            cv2.imwrite ( blob_image_path, blob_image )

    
        for blob_index, blob_image in enumerate ( blob_sequences_for_second_image [ index ] ):

            blob_image_name = "blob" + str(blob_index) + ".jpg"
            
            blob_image_path = os.path.join ( second_image_blob_sequence_folder_path, 
                                                        blob_image_name )
            cv2.imwrite ( blob_image_path, blob_image )
    print ( "Done :)" )


In [28]:
def folder_save_matching_keypoints_blob_sequences ( folder_to_save : str, 
                                            folder_containing_image_pairs : str,
                                            generate_keypoints_func : str,
                                            generate_blob_sequence_func = get_blob_sequence_at_point ) -> None:
    """
        Functionality : 
        - Saves blob sequence consisting of blobs of various sizes having specified keypoints as center
        for images of image pairs in the specified folder. 
        
        Parameters : 
        - folder_to_save : folder path to save blob_sequence.
        - folder_containing_image_pairs : folder path containing image pairs.
        - generate_keypoints_func : function used to generate keypoints.
        - generate_blob_sequence_func : function used to generate blob sequence.
    """
    if not os.path.exists ( folder_to_save ): os.makedirs ( folder_to_save )

    for index, image_pair_folder in enumerate ( os.listdir ( folder_containing_image_pairs ) ):
        
        image_pair_folder_path = os.path.join ( folder_containing_image_pairs, image_pair_folder )
        
        # image_pair_folder_to_save_blob_sequence = "image_pair" + str(index)
        
        # image_pair_folder_to_save_blob_sequence_path = os.path.join ( folder_to_save,
        #                                                             image_pair_folder_to_save_blob_sequence )
        
        # if not os.path.exists ( image_pair_folder_to_save_blob_sequence_path ):
        #     os.makedirs ( image_pair_folder_to_save_blob_sequence_path )

        extended_name_of_blob_sequence_folder = "_image_pair" + str(index)

        save_matching_keypoints_blob_sequences ( folder_to_save = folder_to_save,
                                                image_pair_folder = image_pair_folder_path,
                                                generate_blob_sequence_func = generate_blob_sequence_func,
                                                generate_keypoints_func = generate_keypoints_func,
                                                extended_name_of_blob_sequence_folder = \
                                                    extended_name_of_blob_sequence_folder)

    print ( "Done :)" )


In [29]:
def balance_dataset ( root_dir : str ) -> None:
    """ 
        Balances dataset consisting of two classes by : 
        - removing additional files from class containing maximum number of files if difference 
        between number of files in both classes is less than 10. 
    """
    class_folder_paths = [ os.path.join ( root_dir, class_folder_name ) \
                          for class_folder_name in os.listdir ( root_dir ) ]
    
    class_folder_and_number_of_files = { class_folder_path : len(os.listdir( class_folder_path ))\
                                        for class_folder_path in class_folder_paths }
    
    sorted_class_folder_list = sorted ( class_folder_and_number_of_files, 
                                       key = class_folder_and_number_of_files.get )
    
    diff_in_number_of_files_in_classes = len ( os.listdir ( sorted_class_folder_list[1] ) ) - len ( os.listdir ( sorted_class_folder_list[0] ) )

    if diff_in_number_of_files_in_classes < 20:
        
        majority_class_folder_path = sorted_class_folder_list [ 1 ]
        
        folders_in_majority_class =\
            [ os.path.join ( majority_class_folder_path, sub_folder ) \
            for sub_folder in os.listdir ( majority_class_folder_path )]
    
        folders_to_delete = random.sample(folders_in_majority_class, 
                                        min( diff_in_number_of_files_in_classes, 
                                            len(folders_in_majority_class)))

        print ( len ( folders_to_delete ) )

        for folder in folders_to_delete:
            folder_path = os.path.join( majority_class_folder_path , folder)
            shutil.rmtree ( folder_path )
                

In [30]:
def get_dissimilarity_scores_for_blob_sequences ( blob_sequence_path : str ) -> list:
    """ 
        Returns a numpy array of dissimilarity scores between each blob in the sequence.
    """
    image_paths = [ os.path.join ( blob_sequence_path, image_path )\
                    for image_path in os.listdir ( blob_sequence_path ) ]

    blob_sequence_for_image1 = [ os.path.join ( image_paths[0], blob_path )\
                                    for blob_path in os.listdir( image_paths[0] ) ]
    
    blob_sequence_for_image2 = [ os.path.join ( image_paths[1], blob_path )\
                                    for blob_path in os.listdir( image_paths[1] ) ]
    
    return np.array ( [ get_dissimilarity_score_for_two_images ( blob1, blob_sequence_for_image2 [ index ] )\
                       for index, blob1 in enumerate ( blob_sequence_for_image1 )  ] )

In [31]:
class custom_blob_sequence_dataset ( Dataset ):
    
    def __init__(self, root_dir : str, transform = None) -> None:
        self.root_dir = root_dir

    def __len__ ( self ) -> int:
        
        class_folders = [ os.path.join ( self.root_dir, class_folder ) \
                        for class_folder in os.listdir ( self.root_dir ) ]
        
        return len ( os.listdir ( class_folders [ 0 ] ) ) + len ( os.listdir ( class_folders [ 1 ] ) )
    
    def __getitem__(self, index):
        class_folders = [ os.path.join ( self.root_dir, class_folder ) \
                        for class_folder in os.listdir ( self.root_dir ) ]
        
        blob_sequences = sorted([ os.path.join ( class_folder, blob_sequence_folder ) \
                            for class_folder in class_folders \
                            for blob_sequence_folder in os.listdir ( class_folder ) ])
        
        labels = [ 0 if "dissimilar" == blob_sequence.split("\\")[-2] else 1 \
                for blob_sequence in blob_sequences  ]

        return get_dissimilarity_scores_for_blob_sequences ( blob_sequences[ index ] ), labels [ index ]



In [32]:
def split_into_training_and_testing_folders ( root_dir : str, train_test_split = 0.15 ) -> None:
    """ 
        Splits the root directory into train and test folders.
    """
    
    train_folder = os.path.join ( root_dir , "train" )
    test_folder = os.path.join ( root_dir , "test" )

    for class_folder in os.listdir ( root_dir ):

        if class_folder == "train" or class_folder == "test":
            continue

        if not os.path.exists ( train_folder ): os.makedirs ( train_folder )
        if not os.path.exists ( test_folder ): os.makedirs ( test_folder )

        train_class_folder = os.path.join ( train_folder , class_folder )
        test_class_folder = os.path.join ( test_folder , class_folder )

        if not os.path.exists ( train_class_folder ):
            os.makedirs ( train_class_folder )

        if not os.path.exists ( test_class_folder ):
            os.makedirs ( test_class_folder )

        class_folder_path = os.path.join ( root_dir, class_folder )

        blob_sequence_folder_paths = [ os.path.join ( class_folder_path, blob_sequence_folder ) \
                                    for blob_sequence_folder in os.listdir ( class_folder_path ) ]
        
        train_test_split_index = int ( len ( blob_sequence_folder_paths ) * train_test_split )
        
        test_data = blob_sequence_folder_paths [ : train_test_split_index ]
        train_data = blob_sequence_folder_paths [ train_test_split_index : ]

        for train_data_path in train_data:
            shutil.move ( train_data_path , train_class_folder )
        
        for test_data_path in test_data:
            shutil.move ( test_data_path , test_class_folder  )

In [33]:
train_directory = r"C:\Users\97433\Mafkin_Robotics_FrontEnd\VisualOdometry\feature_extraction\dataset_for_ANN\train"
test_directory = r"C:\Users\97433\Mafkin_Robotics_FrontEnd\VisualOdometry\feature_extraction\dataset_for_ANN\test"

train_DS = custom_blob_sequence_dataset ( root_dir = train_directory )
test_DS = custom_blob_sequence_dataset ( root_dir = test_directory )

train_loader = DataLoader ( train_DS , batch_size = 4 , shuffle = True )
test_loader = DataLoader ( test_DS , batch_size = 4 , shuffle = True  )


In [40]:
class SequenceClassifier ( nn.Module ) :
    def __init__ ( self ) :
        super ( SequenceClassifier , self ).__init__()
        self.fc1 = nn.Linear ( 4 , 16 ).float ()
        self.fc2 = nn.Linear ( 16 , 8 ).float ()
        self.fc3 = nn.Linear ( 8 , 1 ).float ()

    def forward(self, x):
        x = torch.relu ( self.fc1( x ) )
        x = torch.relu ( self.fc2( x ) )
        x = torch.sigmoid( self.fc3( x ) )  
        return x

In [47]:
model = SequenceClassifier()

criterion = nn.BCELoss()  
optimizer = optim.Adam ( model.parameters() , lr = 0.001 )

epochs = 1000
losses = []
for epoch in range(epochs):
    epoch_loss = 0.0
    for inputs, labels in train_loader:
        print ( inputs.shape, labels.shape )

        labels = labels.view( -1 , 1 ).float()
        outputs = model ( inputs.float() )
        loss = criterion( outputs , labels )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * inputs.size(0)

    epoch_loss /= len(train_loader.dataset)
    losses.append(epoch_loss)

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}')

plt.plot(losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.grid(True)
plt.savefig('training_loss_plot.png')
plt.show()

torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) torch.Size([4])
torch.Size([4, 4]) t

KeyboardInterrupt: 